# GOES-R Level 1b Single Band Ingestion

## About

This notebook shows an example of ingesting multiple NetCDF files into a single TileDB group using TileDB-CF-Py. It can be a jumping point for constructing a GOES-R dataset tuned to your specific use case, or just a useful example for understanding the `NetCDF4ConverterEngine`.

## Requirements

### Data

This notebook uses GOES-R L1b radiance data that has been downloaded locally to a specific folder in the project repository. See the script [data/fetch_data.sh](data/fetch_data.sh) for the path hard-coded in this example.

### Software

This notebook was tested with Python 3.9.0, but should work with any Python version that is supported by all required libraries. It requires the following Python libraries:

* `tiledb`
* `tiledb-cf`
* `numpy`
* `netCDF4`

In [ ]:
import glob

import tiledb
import tiledb.cf
import numpy as np

## Find files and auto-generate converter

In this example, we use the TileDB-CF NetCDF4ConverterEngine to combine a dataset of files. We will show the example for data from band 1, but feel free to pick a different channel.

We start by setting some variables:

* `band`: The band (channel) of data we want to copy the radiance product for. Must be between 1-12.
* `group_uri`: Location of the new TileDB group. We store it locally, but you can also store it in the cloud! (e.g. `s3://my-bucket` - best if ran from ec2 instance in same region)
* `dim_dtype`: We can pick the data type we want to use for dense arrays. See []() for more information on the different options.

In [ ]:
band = 1
group_uri = f"output/ABI-L1b-RadF-C{band:02.0f}"
dim_dtype = np.dtype("uint32")
print(f"Copy band {band} to location `{group_uri}` using dimension data type `{dim_dtype}`.")

In [ ]:
abi_files = glob.glob(f"data/noaa-goes17/ABI-L1b-RadF/2021/264/18/OR_ABI-L1b-RadF-M6C{band:02.0f}*.nc")
abi_files

Create a converter for the ABI files. Generate from a file, then view and modify.

** Note: If `NetCDF4ConverterEngine` is not found, it probably means you do not have `netCDF4` installed in your current Python environment. Trying installing `netCDF4` and restarting the kernel.

In [ ]:
abi_converter = tiledb.cf.NetCDF4ConverterEngine.from_file(abi_files[0], dim_dtype=dim_dtype)
abi_converter

## Add time dimension and axis label

In [ ]:
abi_converter.add_shared_dim("time", domain=(0, 143), dtype=dim_dtype) # large enough to hold each scan in the day
for array_creator in abi_converter.array_creators():
    array_creator.domain_creator.inject_dim_creator("time", 1)
abi_converter

## Update array for dataset information

In [ ]:
info_array = abi_converter.get_array_creator_by_attr("goes_imager_projection")
info_array.name = "dataset_information"
info_array.domain_creator.remove_dim_creator("__scalars")


## Create TileDB group and write data

In [ ]:
# WARNING! Deleting a folder on your computer. Proceed with caution.
# This will remove any files existing at `group_uri`. It is useful if you would like to create a new instance of the TileDB group.
import shutil

if True:
    shutil.rmtree(group_uri, ignore_errors=True)

Now let's create our new TileDB group!

In [ ]:
abi_converter.create_group(group_uri)

... and (finally) copy some data in.